In [5]:
from typing import List, Dict,Optional

def client_gpu_assignments(clients: List[str], gpu_ids: List[int]) -> Dict[str, List[int]]:
    n_gpus = len(gpu_ids)
    n_clients = len(clients)
    gpu_assignments = {}
    if n_gpus == 0:
        for client in clients:
            gpu_assignments[client] = []

    if 0 < n_gpus <= n_clients:
        for client_id, client in enumerate(clients):
            gpu_index = client_id % n_gpus
            gpu_assignments[client] = [gpu_ids[gpu_index]]
    elif n_gpus > n_clients > 0:
        client_name_map = {}
        for client_id, client in enumerate(clients):
            client_name_map[client_id] = client

        for gpu_index, gpu_id in enumerate(gpu_ids):
            client_id = gpu_index % n_clients
            client = client_name_map[client_id]
            if client not in gpu_assignments:
                gpu_assignments[client] = []
            gpu_assignments[client].append(gpu_ids[gpu_index])
    return gpu_assignments

In [6]:
client_gpu_assignments(clients = ["site-1", "site-2", "site-3"], gpu_ids = [0,1,2,3,4,5])

{'site-1': [0, 3], 'site-2': [1, 4], 'site-3': [2, 5]}

In [8]:

def _run_poc(cmd_type: str, poc_workspace: str, gpu_ids: List[int], excluded: list, white_list=None):
    if white_list is None:
        white_list = []
    package_commands = _build_commands(cmd_type, poc_workspace, excluded, white_list)
    clients = _get_clients(package_commands)
    gpu_assignments: Dict[str, List[int]] = client_gpu_assignments(clients, gpu_ids)
    for package_name, cmd_path in package_commands:
        print(f"'{cmd_type}', package: '{package_name}', executing '{cmd_path}'")
        if package_name == global_packages[SC.FLARE_PROJ_ADMIN]:
            sync_process(cmd_path)
        elif package_name == global_packages[SC.FLARE_SERVER]:
            async_process(cmd_path, None)
        else:
            async_process(cmd_path, gpu_assignments[package_name])



In [11]:
clients = ["site-1", "site-2", "site-3"]
gpu_ids = [0,1,2,3,4,5]
gpu_assignments: Dict[str, List[int]] = client_gpu_assignments(clients, gpu_ids)

In [7]:

def prepare_env(gpu_ids: Optional[List[int]] = None):
    import os
    my_env = None
    if gpu_ids:
        my_env = os.environ.copy()
        if len(gpu_ids) > 0:
            my_env["CUDA_VISIBLE_DEVICES"] = ",".join([str(gid) for gid in gpu_ids])

    if global_packages.get(SC.IS_DOCKER_RUN):
        my_env = os.environ.copy() if my_env is None else my_env
        if gpu_ids and len(gpu_ids) > 0:
            my_env["GPU2USE"] = my_env["CUDA_VISIBLE_DEVICES"]

        my_env["MY_DATA_DIR"] = os.path.join(get_poc_workspace(), "data")

    return my_env


In [16]:
import os
gpu_ids2 = gpu_assignments["site-1"]
my_env = os.environ.copy()
my_env["CUDA_VISIBLE_DEVICES"] = ",".join([str(gid) for gid in gpu_ids2])
my_env["CUDA_VISIBLE_DEVICES"]

'0,3'